<a href="https://colab.research.google.com/github/saurabhgssingh/genre_classifier/blob/master/Multi_Label_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.1.0

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.1.0`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [ ]:
import tensorflow as tf
tf.__version__



'2.3.0'

In [ ]:
import nltk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
from sklearn.metrics import f1_score,precision_score,recall_score
import re

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
movies=pd.read_csv("/content/drive/My Drive/Incubator/movies.csv",usecols=["title",'description','genres'])
movies.head()

,title,description,genres
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","animation, comedy, family"
1,Jumanji,When siblings Judy and Peter discover an encha...,"adventure, fantasy, family"
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"romance, comedy"
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","comedy, drama, romance"
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,comedy


In [ ]:
movies['description'] = movies['description'].astype('str')
all_genres = [s.split(", ") for s in movies[movies.genres.notnull()].genres]
genres = [item for l in all_genres for item in l ]
movies['genres']=movies.genres.str.split(",")
movies['genres'].replace([np.nan],[' '],inplace=True)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

movies['clean_plot'] = movies['description'].apply(lambda x: clean_text(x))

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

movies['clean_plot'] = movies['clean_plot'].apply(lambda x: remove_stopwords(x))
movies = movies [['clean_plot', 'genres']]
from sklearn.preprocessing import MultiLabelBinarizer
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(movies['genres'])



# transform target variable
labels = multilabel_binarizer.transform(movies['genres'])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
embedding_dim = 100
max_length = 200
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size=37303
test_portion=.1


In [ ]:
sentences=list(movies['clean_plot'].astype('str'))

In [ ]:
from sklearn.model_selection import train_test_split
train_sent, test_sent, train_label, test_label = train_test_split(sentences, labels, test_size=0.2, random_state=9)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_sent)
word_index = tokenizer.word_index
vocab_size=len(word_index)

Train sequences and paddings

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sent)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

Test sequences and paddings

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_sent)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
train_padded.shape

(37302, 700)

In [ ]:
!pip install h5py pyyaml

In [ ]:
checkpoint_path="/content/drive/My Drive/Incubator/cp.cpkt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only = True, verbose =1)

# MODEL

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size+1, output_dim=64, input_length=max_length),
                             tf.keras.layers.Conv1D(128, 5, activation='relu'),
                             tf.keras.layers.GlobalAveragePooling1D(),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(41, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 64)           4450880   
_________________________________________________________________
conv1d (Conv1D)              (None, 196, 128)          41088     
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 41)                2665      
Total params: 4,502,889
Trainable params: 4,502,889
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics = ['accuracy'])

In [ ]:
history = model.fit(train_padded, train_label, epochs=10, validation_data=(test_padded, test_label), verbose=2)

Epoch 1/10
1166/1166 - 48s - loss: 0.0572 - accuracy: 0.4415 - val_loss: 0.3244 - val_accuracy: 0.1786
Epoch 2/10
1166/1166 - 50s - loss: 0.0525 - accuracy: 0.4505 - val_loss: 0.3502 - val_accuracy: 0.1799
Epoch 3/10
1166/1166 - 47s - loss: 0.0482 - accuracy: 0.4591 - val_loss: 0.3712 - val_accuracy: 0.1799
Epoch 4/10
1166/1166 - 47s - loss: 0.0445 - accuracy: 0.4679 - val_loss: 0.3985 - val_accuracy: 0.1809
Epoch 5/10
1166/1166 - 47s - loss: 0.0410 - accuracy: 0.4762 - val_loss: 0.4239 - val_accuracy: 0.1792
Epoch 6/10
1166/1166 - 47s - loss: 0.0378 - accuracy: 0.4858 - val_loss: 0.4560 - val_accuracy: 0.1780
Epoch 7/10
1166/1166 - 47s - loss: 0.0348 - accuracy: 0.4888 - val_loss: 0.4831 - val_accuracy: 0.1752
Epoch 8/10
1166/1166 - 47s - loss: 0.0320 - accuracy: 0.4985 - val_loss: 0.5218 - val_accuracy: 0.1747
Epoch 9/10
1166/1166 - 47s - loss: 0.0295 - accuracy: 0.5030 - val_loss: 0.5421 - val_accuracy: 0.1733
Epoch 10/10
1166/1166 - 47s - loss: 0.0273 - accuracy: 0.5044 - val_loss:

In [ ]:
model.save("/content/drive/My Drive/Incubator/model.h5")

# MODEL 2

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt
embeddings_index = {};
with open('/tmp/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

--2020-08-09 15:25:17--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 64.233.187.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/tmp/glove.6B.100d.txt’

/tmp/glove.6B.100d. 100%[===================>] 331.04M  57.3MB/s    in 5.8s    

2020-08-09 15:25:23 (57.3 MB/s) - ‘/tmp/glove.6B.100d.txt’ saved [347116733/347116733]



In [ ]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(41, activation='sigmoid')
])
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 100)          6954500   
_________________________________________________________________
dropout (Dropout)            (None, 200, 100)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 196, 64)           32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 64)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense_2 (Dense)              (None, 41)                2665      
Total params: 7,022,253
Trainable params: 67,753
Non-trainable params: 6,954,500
_______________________________________

In [ ]:
history1 = model1.fit(train_padded, train_label, epochs=25, validation_data=(test_padded, test_label), verbose=2)

Epoch 1/25
1166/1166 - 17s - loss: 0.1839 - accuracy: 0.1066 - val_loss: 0.1747 - val_accuracy: 0.1083
Epoch 2/25
1166/1166 - 15s - loss: 0.1744 - accuracy: 0.1101 - val_loss: 0.1747 - val_accuracy: 0.1083
Epoch 3/25
1166/1166 - 17s - loss: 0.1744 - accuracy: 0.1105 - val_loss: 0.1750 - val_accuracy: 0.1083
Epoch 4/25
1166/1166 - 18s - loss: 0.1743 - accuracy: 0.1106 - val_loss: 0.1750 - val_accuracy: 0.1083
Epoch 5/25
1166/1166 - 18s - loss: 0.1743 - accuracy: 0.1107 - val_loss: 0.1749 - val_accuracy: 0.1083
Epoch 6/25
1166/1166 - 18s - loss: 0.1743 - accuracy: 0.1106 - val_loss: 0.1749 - val_accuracy: 0.1083
Epoch 7/25
1166/1166 - 18s - loss: 0.1743 - accuracy: 0.1105 - val_loss: 0.1752 - val_accuracy: 0.1083
Epoch 8/25
1166/1166 - 18s - loss: 0.1743 - accuracy: 0.1108 - val_loss: 0.1748 - val_accuracy: 0.1083
Epoch 9/25
1166/1166 - 18s - loss: 0.1743 - accuracy: 0.1109 - val_loss: 0.1748 - val_accuracy: 0.1083
Epoch 10/25
1166/1166 - 19s - loss: 0.1742 - accuracy: 0.1109 - val_loss:

In [ ]:
model1.save("/content/drive/My Drive/Incubator/model_glove.h5")

# Loading model

In [ ]:
from tensorflow import keras
model = keras.models.load_model("/content/drive/My Drive/Incubator/my_model.h5")

In [ ]:
def predictor(text):
  text = clean_text(text)
  text = remove_stopwords(text)
  sequences = tokenizer.texts_to_sequences([text])
  padded = pad_sequences(sequences,  maxlen=max_length, padding=padding_type, truncating=trunc_type)
  prediction = model.predict(padded)
  return prediction



In [ ]:
x = movies['clean_plot'][4]

In [ ]:
predictions = model.predict([test_padded])

In [ ]:
predictions[1].shape

(41,)

In [ ]:
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for val in thresholds:
    print ("threshold: ",val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(test_label, pred, average='micro')
    recall = recall_score(test_label, pred, average='micro')
    f1 = f1_score(test_label, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

threshold:  0.1
Micro-average quality numbers
Precision: 0.2193, Recall: 0.3834, F1-measure: 0.2790
threshold:  0.2
Micro-average quality numbers
Precision: 0.2428, Recall: 0.3444, F1-measure: 0.2848
threshold:  0.3
Micro-average quality numbers
Precision: 0.2628, Recall: 0.3167, F1-measure: 0.2872
threshold:  0.4
Micro-average quality numbers
Precision: 0.2779, Recall: 0.2966, F1-measure: 0.2869
threshold:  0.5
Micro-average quality numbers
Precision: 0.2915, Recall: 0.2794, F1-measure: 0.2853
threshold:  0.6
Micro-average quality numbers
Precision: 0.3051, Recall: 0.2595, F1-measure: 0.2805
threshold:  0.7
Micro-average quality numbers
Precision: 0.3195, Recall: 0.2434, F1-measure: 0.2763
threshold:  0.8
Micro-average quality numbers
Precision: 0.3357, Recall: 0.2256, F1-measure: 0.2698
threshold:  0.9
Micro-average quality numbers
Precision: 0.3578, Recall: 0.2020, F1-measure: 0.2582


In [ ]:
s ="Ethan Hunt and the IMF team join forces with CIA assassin August Walker to prevent a disaster of epic proportions. Arms dealer John Lark and a group of terrorists known as the Apostles plan to use three plutonium cores for a simultaneous nuclear attack on the Vatican, Jerusalem and Mecca, Saudi Arabia. When the weapons go missing, Ethan and his crew find themselves in a desperate race against time to prevent them from falling into the wrong hands"
clean = clean_text(s)
stop = remove_stopwords(clean)
token = tokenizer.texts_to_sequences([stop])
pad = pad_sequences(token,  maxlen=max_length, padding=padding_type, truncating=trunc_type)
x = model.predict(pad)

In [ ]:
x>0.1

array([[False, False, False, False, False,  True, False,  True, False,
        False, False, False, False, False, False, False, False,  True,
        False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False]])

In [ ]:
multilabel_binarizer.inverse_transform((x>0.2).astype(np.int))

[(' adventure', ' thriller', 'action')]

In [ ]:
import pickle

In [ ]:
with open('/content/drive/My Drive/Incubator/tokenizer1.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
with open('/content/drive/My Drive/Incubator/binarizer.pickle', 'wb') as handle:
    pickle.dump(multilabel_binarizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('/content/drive/My Drive/Incubator/binarizer.pickle', 'rb') as handle:
    multi_label = pickle.load(handle)

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
class clean_text():
      def __init__(self, text = "test"):
        self.text = text

      def remove_stopwords(self):
          no_stopword_text = [w for w in self.text.split() if not w in stop_words]
          self.text = ' '.join(no_stopword_text)
          return self
  
      def clean(self):
          # remove backslash-apostrophe 
          txt = re.sub("\'", "", self.text) 
          # remove everything except alphabets 
          txt = re.sub("[^a-zA-Z]"," ",txt) 
          # remove whitespaces 
          txt = ' '.join(txt.split()) 
          # convert text to lowercase 
          self.text = txt.lower() 
          return self
      
      def do_all(self,text):
        self.text = text
        self = self.remove_stopwords()
        self = self.clean()
        return self.text

In [ ]:
clean = clean_text()

In [ ]:
import pickle

In [ ]:
with open('/content/drive/My Drive/Incubator/cleaner1.pickle', 'wb') as handle:
    pickle.dump(cleat, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('/content/drive/My Drive/Incubator/cleaner.pickle', 'rb') as handle:
   cleand = pickle.load(handle)

In [ ]:
with open('/content/drive/My Drive/Incubator/cleaner1.pickle', 'rb') as handle:
    dum = pickle.load(handle)

In [ ]:
text = "vnsfjdkvniofnvioernvn  infoiwejocp nejfojwjfjj99wfj jciowjc"
cleand.do_all(text)

'vnsfjdkvniofnvioernvn infoiwejocp nejfojwjfjj wfj jciowjc'

In [ ]:
import sklearn
tf.__version__

'2.3.0'